In [29]:
import torch
import torch.nn as nn
from collections import deque
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from importlib import reload

# local imports
from rails import rails as r
from rails import aise as a
import model as _model
import utils as _tools

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
"""
initialize the trainned model
"""

resnet = _model.TransferModel(use_cpu=False)

# model = VGG()
# model = resnet.model

In [4]:
"""
load the saved state
"""
resnet.model.load_state_dict(torch.load(
    "models/Cardiomegaly_resnet18.pth", map_location=DEVICE
))

resnet.model.eval()

print("done.")

done.


In [5]:
# the training data from the data loader
n_rows = 100 # how many rows to sample
row_count = 0
x = []
y = []
for image, labels in resnet.dataloader_train.dataset:
    row_count += 1
    x.append(image)
    y.append(labels[0]) # the label for cadiomegaly
    
    if row_count == n_rows:
        break
    
    
x_train = torch.FloatTensor(x)
y_train = torch.LongTensor(y)

print("done")

done


<ipython-input-5-43305d3bf59b>:27: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y_train = torch.LongTensor(y)


In [6]:
x_train.shape

torch.Size([100, 1, 224, 224])

In [9]:
rails

In [44]:
reload(r)
reload(a)

CONFIG = {
    "start_layer": 1,
    "n_class": 2,
    "aise_params": [
        {"hidden_layer": 2, 
         "sampling_temperature": 1, 
         "max_generation": 10, 
         "n_neighbors" : 3,
         "n_population" : 2 * 3,
         "mut_range": (.005, .015)},
    ]
}

rails = r.RAILS(resnet.model, 
              CONFIG, 
              x_train,
              y_train)

Cannot build query objects!
